In [53]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from matplotlib.animation import FuncAnimation

class Rom():
    def __init__(self, T, n, legemer):
        self.T = T
        self.n = n
        self.t = np.linspace(0,T,n)
        self.dt = T/n
        self.legemer = legemer
        self.legemer_pos = []
        self.antall = 0
        self.farger = ['blue', 'orange', 'red', 'green', 'black', 'magenta', 'cyan', 'purple', 'pink', 'olive']
        
    def Fg(self,r,m,R,M):
        d_vektor = r - R
        d_abs = np.linalg.norm(d_vektor)
        F = -6.67*10**(-11) * m*M/d_abs**3 * d_vektor
        return F
        
    def oppdaterLegeme(self, legeme, i):
        ny_legemer = self.legemer.copy()
        ny_legemer.remove(legeme)
        a = np.sum([self.Fg(legeme.pos, legeme.masse, x.pos, x.masse)[0] for x in ny_legemer])
        b = np.sum([self.Fg(legeme.pos, legeme.masse, x.pos, x.masse)[1] for x in ny_legemer])
        Fp = np.array([a,b])
        ap = Fp/legeme.masse
        legeme.pos = legeme.pos + legeme.fart * self.dt
        legeme.fart = legeme.fart + ap *self.dt
        legeme.xverdier[i], legeme.yverdier[i] = legeme.pos[0], legeme.pos[1]
        
    def simulerRom(self):
        for i in range(self.n):
            for legeme in self.legemer:
                self.oppdaterLegeme(legeme, i)

    def plot(self):
        #Tegner banene.
        plt.figure(1)
        #Legemene.
        for legeme in self.legemer:
            farge = self.farger[self.legemer.index(legeme)]
            plt.plot(legeme.xverdier, legeme.yverdier, ls = '--', color=farge)
            if legeme.stjerne == True:
                plt.plot(legeme.pos[0], legeme.pos[1], marker='*', markersize=20, color=farge)
            else:
                plt.plot(legeme.pos[0], legeme.pos[1], marker='o', markersize=10, color=farge)
        plt.axis('equal')
        plt.grid()
        plt.show()
    
    def animate(self, frame):
        for legeme_pos in self.legemer_pos:
            i = self.legemer_pos.index(legeme_pos)
            legeme = self.legemer[i]
            legeme_pos.set_data((legeme.xverdier[int(n*frame/self.antall)],legeme.yverdier[int(n*frame/self.antall)]))
        
    def animer(self):
        fig,ax=plt.subplots()
        plt.axis('equal')
        plt.xlim([-1E12,2E12])
        plt.ylim([-3E12,2E12])
        for legeme in self.legemer:
            farge = self.farger[self.legemer.index(legeme)]
            if legeme.stjerne == True:
                Legeme_plot = plt.plot([],color=farge, marker='*', markersize=10) #lager et punktplot, der posisjonen er tom
            else:
                planetplot=plt.plot([], color=farge, marker='o', markersize=10)
            legeme_pos = planetplot[0] # klargjør at det er posisjonen i plottet som skal oppdateres/animeres
            self.legemer_pos.append(legeme_pos)
        time_template = 'tid = %.0f dager'
        tekst = plt.text(0.05, 0.9, '', transform=ax.transAxes)
        
        self.antall=200 #antall bilder som animeres totalt
        anime=FuncAnimation(fig,self.animate,frames=self.antall, interval=50)
        plt.close()

        HTML(anime.to_jshtml())

class Legeme():
    def __init__(self, start_pos, start_fart, masse, stjerne = False):
        self.pos = start_pos
        self.fart = start_fart
        self.masse = masse
        self.xverdier = np.zeros(n)
        self.yverdier = np.zeros(n)
        self.stjerne = stjerne

In [54]:
n = 10000
#P1 = Legeme(np.array([-2.2*10**11, 0]), np.array([0, -1.0*10**3]), 6.4*10**24, n)
#S1 = Legeme(np.array([4.5*10**11, 3.0*10**11]), np.array([0, -7.5*10**3]), 8.0*10**30, n, True)
#S2 = Legeme(np.array([0, 0]), np.array([0, -3.0*10**4]), 2.0*10**30, n, True)
Jorden = Legeme(np.array([1.5*10**11, 0]), np.array([0, 2.98*10**4]), 5.97*10**24)
Solen = Legeme(np.array([0, 0]), np.array([0, 0]), 1.989*10**30, True)
Månen = Legeme(np.array([1.5*10**11+3.9*10**8, 0]), np.array([0, 2.98*10**4+1010]), 7.35*10**22)
legemer = [Jorden, Solen, Månen]

rom = Rom(1*365*24*60*60, n, legemer)
rom.simulerRom()
rom.animer()